In [6]:
import numpy as np
import sys
sys.path.append("src/")
import pygad
from calcu_lambda import compute_lambda
from functools import partial
from soft_thres import soft_thres_l1
import sys



# Define the objective function
def fit_func(W, v, lambda_, ga_instance, solution, solution_idx):
    ###!! Calculating lambda, this method is only valid when no PPI is considered

    # mean_v = np.mean(v)
    # sd_v = np.std(v)

    # # for W, calculate row mean
    # mean_W = np.mean(W)
    # sd_W = np.std(W)
    # M_v = mean_v / sd_v
    # M_w = mean_W / sd_W
    # lambda_ = M_w / (M_v + M_w)
    # print(f"Lambda: {lambda_}")

    x = np.array(solution)

    term1 = lambda_ * np.dot(x.T, np.dot(W, x))
    term2 = (1 - lambda_) * np.dot(v.T, x)
    loss = term1 + term2
    return loss

# def on_mutation(ga_instance):
#     for sol_idx, solution in enumerate(ga_instance.population):
#         # Ensure non-negativity
#         solution = np.maximum(solution, 0)
#         solution = soft_thres_l1(solution)
#         # solution = solution / np.sum(solution)
#         ga_instance.population[sol_idx] = solution
#         print(solution.sum(), solution.max(), 'mutation')

def on_mutation(ga_instance, offspring):
    for sol_idx in range(offspring.shape[0]):
        # Ensure non-negativity
        offspring[sol_idx] = np.maximum(offspring[sol_idx], 0)
        # Apply soft thresholding for L1 regularization
        offspring[sol_idx] = soft_thres_l1(offspring[sol_idx])
    return offspring

def on_generation(ga_instance):
    for sol_idx, solution in enumerate(ga_instance.population):
        # Ensure non-negativity
        # solution = np.maximum(solution, 0)
        # # solution = solution / np.sum(solution)
        # ga_instance.population[sol_idx] = solution
        solution = soft_thres_l1(solution)
    print(
        f"Generation {ga_instance.generations_completed}: Best Fitness = {ga_instance.best_solution()[1]}"
    )


def create_normalized_population(sol_per_pop, num_genes):
    population = np.random.uniform(
        low=0.0, high=1.0, size=(sol_per_pop, num_genes)
    )
    # population /= population.sum(axis=1, keepdims=True)
    return population


def arithmetic_crossover(parents, offspring_size, ga_instance):
    offspring = np.empty(offspring_size)
    for k in range(offspring_size[0]):
        parent1_idx = k % parents.shape[0]
        parent2_idx = (k + 1) % parents.shape[0]
        alpha = np.random.uniform(0, 1, size=offspring_size[1])
        offspring[k, :] = alpha * parents[parent1_idx, :] + (1 - alpha) * parents[parent2_idx, :]
    return offspring


if __name__ == "__main__":
    for idx in range(1, 2):
        W = np.load(f"output/sparse_v_W/W_case{idx}.npy")
        v = np.load(f"output/sparse_v_W/v_case{idx}.npy")
        lambda_ = compute_lambda(W, v, num_samples=1000)
        # print(f"Lambda: {lambda_}")
        fitness_func = lambda ga_instance, solution, solution_idx: fit_func(
            W, v, lambda_, ga_instance, solution, solution_idx
        )

        # GA parameters
        num_iterations = 1000  # Set the number of iterations # 60000
        num_generations = (
            num_iterations  # Assuming one generation per iteration
        )
        num_parents_mating = 50
        sol_per_pop = 100
        num_genes = len(W)  # Ensure num_genes matches the dimension of W and v
        mutation_rate = 1 / (num_genes + 1)  # Set mutation rate
        crossover_rate = 0.5  # Set crossover rate
        initial_population = create_normalized_population(
            sol_per_pop, num_genes
        )

        # Creating an instance of the GA
        ga_instance = pygad.GA(
            num_generations=num_generations,
            num_parents_mating=num_parents_mating,
            fitness_func=fitness_func,
            sol_per_pop=sol_per_pop,
            num_genes=num_genes,
            mutation_type="random",
            mutation_percent_genes=20,  # pygad uses percentage
            on_mutation=on_mutation,
            crossover_type=arithmetic_crossover,  # Experiment with different crossover methods
            crossover_probability=crossover_rate,  # Set crossover rate
            on_generation=on_generation,
            stop_criteria=[
                "saturate_10"
            ],  # Stop if no improvement for 100 generations
            keep_parents=1,
            initial_population=initial_population,
        )
        # Running the GA
        ga_instance.run()

        # Best solution
        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        print("Best Solution: ", solution)
        print("Best Solution Fitness: ", solution_fitness)

        # Save the best solution
        # np.save(f"output/best_solution_case{idx}.npy", solution)
        
    

Generation 1: Best Fitness = 6828.164643662153
Generation 2: Best Fitness = 6828.164643662153
Generation 3: Best Fitness = 6828.164643662153
Generation 4: Best Fitness = 6828.164643662153
Generation 5: Best Fitness = 6828.164643662153
Generation 6: Best Fitness = 6828.164643662153
Generation 7: Best Fitness = 6828.164643662153
Generation 8: Best Fitness = 6828.164643662153
Generation 9: Best Fitness = 6828.164643662153
Generation 10: Best Fitness = 6828.164643662153
Best Solution:  [0.89040668 0.0858273  0.70171714 0.42529727 0.01559578 0.70170707
 0.00751858 0.50395963 0.7386455  0.52880835 0.95663149 0.3614837
 0.12352611 0.65897753 0.72885246 0.8553211  0.89151823 0.24109868
 0.65944656 0.03441046 0.79934045 0.62143353 0.06843768 0.03837044
 0.53348802 0.58148843 0.25611836 0.35704658 0.1837005  0.14306727
 0.80801487 0.40019292 0.04718437 0.36709247 0.00312793 0.49868036
 0.75898044 0.35463695 0.81386353 0.5235866  0.52126594 0.66061901
 0.1782455  0.86372884 0.3905496  0.60937561 

In [25]:
print(solution.sum())

1.0


In [26]:
solution_soft = soft_thres_l1(solution)
print("Soft-thresholded Solution: ", solution_soft.sum())
# print non-zero elements as list
print("Non-zero elements: ", np.nonzero(solution_soft)[0])
indices = np.nonzero(solution_soft)[0]

Soft-thresholded Solution:  1.0
Non-zero elements:  [  0   1   2   3   4   5   7   8   9  10  11  12  13  14  16  17  18  19
  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36  37
  38  39  40  41  42  43  44  45  46  47  48  49  51  52  53  54  55  56
  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74
  75  76  77  78  79  80  81  82  83  84  86  87  88  89  90  91  92  93
  94  95  96  97  98  99 100 101 103 104 105 106 107 108 109 110 111 112
 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130
 131 132 133 134 135 136 137 138 139 140 141 142 144 145 146 147 148 149
 150 151 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168
 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187
 188 189 190 191 192 193 195 196 197 198 199 200 201 202 203 204 205 206
 207 208 209 211 213 214 215 216 217 218 219 220 222 223 224 225 226 227
 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245

In [27]:
print(len(indices))

947


In [7]:
for idx in range(1, 5):
    W = np.load(f"output/sparse_v_W/W_case{idx}.npy")
    v = np.load(f"output/sparse_v_W/v_case{idx}.npy")

    print(W.sum(), v.sum())

94305.56531245612 902.705065424358
93240.590645145 579.1497630157643
117975.8971528779 897.5093933807915
83994.81533151757 578.9871866849234


In [6]:
import numpy as np
import pygad

from functools import partial


# Define the objective function
def fit_func(W, v, ga_instance, solution, solution_idx):
    ###!! Calculating lambda, this method is only valid when no PPI is considered

    mean_v = np.mean(v)
    sd_v = np.std(v)

    # for W, calculate row mean
    mean_W = np.mean(W)
    sd_W = np.std(W)
    M_v = mean_v / sd_v
    M_w = mean_W / sd_W
    lambda_ = M_w / (M_v + M_w)
    # print(f"Lambda: {lambda_}")

    x = np.array(solution)

    term1 = lambda_ * np.dot(x.T, np.dot(W, x))
    term2 = (1 - lambda_) * np.dot(v.T, x)
    loss = term1 + term2
    return loss


def on_generation(ga_instance):
    for sol_idx, solution in enumerate(ga_instance.population):
        # Ensure non-negativity
        solution = np.maximum(solution, 0)
        # solution = solution / np.sum(solution)
        ga_instance.population[sol_idx] = solution
    print(
        f"Generation {ga_instance.generations_completed}: Best Fitness = {ga_instance.best_solution()[1]}"
    )


if __name__ == "__main__":
    for idx in range(1, 5):
        W = np.load(f"output/sparse_v_W/W_case{idx}.npy")
        v = np.load(f"output/sparse_v_W/v_case{idx}.npy")
        fitness_func = lambda ga_instance, solution, solution_idx: fit_func(
            W, v, ga_instance, solution, solution_idx
        )

        # GA parameters
        num_iterations = 10000  # Set the number of iterations # 60000
        num_generations = (
            num_iterations  # Assuming one generation per iteration
        )
        num_parents_mating = 10
        sol_per_pop = 20
        num_genes = len(W)  # Ensure num_genes matches the dimension of W and v
        mutation_rate = 1 / (num_genes + 1)  # Set mutation rate
        crossover_rate = 0.5  # Set crossover rate

        # Creating an instance of the GA
        ga_instance = pygad.GA(
            num_generations=num_generations,
            num_parents_mating=num_parents_mating,
            fitness_func=fitness_func,
            sol_per_pop=sol_per_pop,
            num_genes=num_genes,
            mutation_type="random",
            mutation_percent_genes=20,  # pygad uses percentage
            crossover_type="single_point",  # Experiment with different crossover methods
            crossover_probability=crossover_rate,  # Set crossover rate
            on_generation=on_generation,
            stop_criteria=[
                "saturate_10"
            ],  # Stop if no improvement for 100 generations
            keep_parents=1,
            initial_population=np.random.uniform(
                low=0.0, high=1.0, size=(sol_per_pop, num_genes)
            ),  # Ensure initial population in [0,1]
        )
        # Running the GA
        ga_instance.run()

        # Best solution
        solution, solution_fitness, solution_idx = ga_instance.best_solution()
        print("Best Solution: ", solution)
        print("Best Solution Fitness: ", solution_fitness)

        # Save the best solution
        # np.save(f"output/best_solution_case{idx}.npy", solution)


Generation 1: Best Fitness = 5952.138115521779
Generation 2: Best Fitness = 6630.847684066706
Generation 3: Best Fitness = 7615.634165057199
Generation 4: Best Fitness = 7689.028809278906
Generation 5: Best Fitness = 8295.278307563693
Generation 6: Best Fitness = 9949.885996257235
Generation 7: Best Fitness = 11170.969799782999
Generation 8: Best Fitness = 11949.87872472393
Generation 9: Best Fitness = 13299.682450349908
Generation 10: Best Fitness = 14898.237021559504
Generation 11: Best Fitness = 15785.57463174141
Generation 12: Best Fitness = 18700.673232161786
Generation 13: Best Fitness = 19050.044773053876
Generation 14: Best Fitness = 20663.95041225535
Generation 15: Best Fitness = 22608.11857038621
Generation 16: Best Fitness = 26015.54778445916
Generation 17: Best Fitness = 26861.043214659992
Generation 18: Best Fitness = 28741.560193721987
Generation 19: Best Fitness = 30604.616890483612
Generation 20: Best Fitness = 33812.36588017229
Generation 21: Best Fitness = 34644.21257

KeyboardInterrupt: 

In [9]:
import numpy as np

# Load your W matrix
W = np.load("output/v_W/W_case2.npy")

rtol = 1e-02  # relative tolerance
atol = 1e-03  

if np.allclose(W, W.T, rtol=rtol, atol=atol):
    print("W is symmetric.")

else:
    print("W is not symmetric.")

#print to 3 s.f.
np.set_printoptions(precision=5)
print(W[:5, :5])
print("=============")
print(W.T)



W is not symmetric.
[[2.36792e-17 1.25733e+01 4.58668e-01 4.62585e-01 5.45183e-01]
 [3.35846e+00 2.39439e-16 3.35663e+00 3.35580e+00 3.35369e+00]
 [1.55892e-01 9.18533e+00 1.80846e-18 1.52127e-01 1.29038e-01]
 [1.31500e-01 1.11469e+01 1.25391e-01 1.18172e-16 1.68509e-01]
 [6.64737e-02 1.69014e+00 5.20587e-02 5.60532e-02 9.82460e-18]]
[[2.36792e-17 3.35846e+00 1.55892e-01 ... 4.52723e-02 3.09185e-01
  0.00000e+00]
 [1.25733e+01 2.39439e-16 9.18533e+00 ... 1.94375e+01 2.11762e+01
  0.00000e+00]
 [4.58668e-01 3.35663e+00 1.80846e-18 ... 5.87584e-02 4.12739e-01
  0.00000e+00]
 ...
 [4.54572e-01 3.35550e+00 1.61108e-01 ... 1.83586e-17 3.88158e-01
  0.00000e+00]
 [3.88630e-01 3.35938e+00 1.98867e-01 ... 6.68339e-02 1.54275e-17
  0.00000e+00]
 [7.63562e-01 3.33906e+00 1.68955e-01 ... 2.22946e-01 3.24079e-01
  0.00000e+00]]


In [2]:
import numpy as np
from scipy.stats import multivariate_normal
import pandas as pd


if __name__ == "__main__":
    # Set the parameters
    num_genes = 10  # Number of variables (genes)
    num_samples = 5  # Number of samples
    data_range = (-0.5, 0.5)

    k = 0.6
    r = 0.65

    # Generate means from a uniform distribution
    means = np.random.uniform(
        low=data_range[0], high=data_range[1], size=num_genes
    )
    # Generate the standard deviation matrix
    std_dev = np.zeros((num_genes, num_genes))
    np.fill_diagonal(
        std_dev,
        np.random.uniform(low=0.01, high=data_range[1], size=num_genes),
    )

    # Initialize the correlation matrix
    corr_matrix = np.eye(num_genes)  # with zeros

    # Define the significant genes
    l = np.random.choice(
        range(num_genes), size=int(0.2 * num_genes), replace=False
    )

    # Generate the modified means and correlations for significant genes
    for i in l:
        means[i] += k
        for j in l:
            if i != j:
                corr_matrix[i, j] += r

    # cov matrix is corr matrix_ih * std_i * std_h
    cov_matrix = std_dev.dot(corr_matrix).dot(std_dev)

    # Generate the simulated expression data
    simulated_data = multivariate_normal.rvs(
        mean=means, cov=cov_matrix, size=num_samples
    )

    # Convert the numpy array to a pandas DataFrame
    df = pd.DataFrame(simulated_data)

    # Save the DataFrame to a CSV file
    df.to_csv("simulated_data_test.csv", index=False)

    print("Data saved to simulated_data.csv")

    print(simulated_data.shape)


Data saved to simulated_data.csv
(5, 10)


In [3]:
print("means", means)
print("corr_matrix", corr_matrix)
print("cov_matrix", cov_matrix)


means [ 0.36124791 -0.12239389  1.0872388   0.24336767 -0.18461028  0.38343517
 -0.10862884  0.12641376  0.11064644  0.16519982]
corr_matrix [[1.   0.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   1.   0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   1.   0.   0.   0.   0.   0.   0.65 0.  ]
 [0.   0.   0.   1.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   1.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   1.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   1.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   1.   0.   0.  ]
 [0.   0.   0.65 0.   0.   0.   0.   0.   1.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]
cov_matrix [[0.08631339 0.         0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.23201425 0.         0.         0.         0.
  0.         0.         0.         0.        ]
 [0.         0.         0.053856   0.         0.         0.
  0.         0.         0.01432101 0.      